In [1]:
import math
import sympy
import mpmath
import random
import primefac
import numpy as np
import scipy.linalg
import qutip
import vpython

def scalar_to_vector(z):
    x = z.real
    y = z.imag
    return [(2*x)/(1.+(x**2)+(y**2)), \
           (2*y)/(1.+(x**2)+(y**2)), \
           (-1.+(x**2)+(y**2))/(1.+(x**2)+(y**2))]

def vector_to_scalar(x, y, z):
    if z == 1:
        return Inf 
    else:
        return complex(x/(1-z), y/(1-z))

class Sphere:
    def __init__(self, n, dt=0.005):
        self.t = 0
        self.n = n
        self.dt = dt
        self.state = qutip.rand_ket(self.n)
        self.energy = qutip.rand_herm(self.n)
        self.mobius = np.eye(2)
        self.dims = None
        self.polynomial = None
        self.pure_scalars = None
        self.pure_vectors = None
        self.pure_matrices = None
        self.pure_energy = None
        self.impure_vectors = None
        self.impure_matrices = None
        self.impure_energies = None
        self.shatter()

    def shatter(self):
        self.polynomial = self.state.full().T.tolist()[0]
        self.pure_scalars = mpmath.polyroots(self.polynomial)
        self.pure_vectors = [scalar_to_vector(scalar) for scalar in self.pure_scalars]
        self.pure_matrices = [vector[0]*qutip.sigmax().full() + \
                              vector[1]*qutip.sigmay().full() + \
                              vector[2]*qutip.sigmaz().full() 
                                for vector in self.pure_vectors]
        self.pure_energy = qutip.expect(self.energy, self.state)
        pfacs = list(primefac.primefac(self.n))
        twos = pfacs.count(2)
        remainder = int(self.n/2**twos)
        splitting = [2]*twos
        if remainder != 1:
            self.state.dims = [splitting+[remainder],[1]*(len(splitting)+1)]
        else:
            self.state.dims = [splitting,[1]*(len(splitting))]
        self.dims = self.state.dims[0]
        self.impure_matrices = [self.state.ptrace(i) for i in range(len(splitting))]
        self.energy.dims = [self.dims, self.dims]
        self.impure_energies = [self.energy.ptrace(i) for i in range(len(splitting))]
        self.energy.dims = [[self.n], [self.n]]
        self.impure_energies = [qutip.expect(self.impure_energies[i], self.impure_matrices[i]) for i in range(len(self.impure_matrices))]
        self.impure_vectors = [[qutip.expect(qutip.sigmax(), matrix), \
                                qutip.expect(qutip.sigmay(), matrix), \
                                qutip.expect(qutip.sigmaz(), matrix)] 
                                    for matrix in self.impure_matrices]
    
    def mobius_transform(self):
        print('pure_matrices')
        print(self.pure_matrices)
        propagator = scipy.linalg.expm(-2*math.pi*complex(0,1)*self.mobius)
        for i in range(len(self.pure_matrices)):
            self.pure_matrices[i] = np.conjugate(propagator).T*self.pure_matrices[i]*propagator
        print('pure matrices after')
        print(self.pure_matrices)
        print("pure vectors")
        self.pure_vectors = [[qutip.expect(qutip.sigmax(), qutip.Qobj(matrix)), \
                                qutip.expect(qutip.sigmay(), qutip.Qobj(matrix)), \
                                qutip.expect(qutip.sigmaz(), qutip.Qobj(matrix))] 
                                    for matrix in self.pure_matrices]
        print(self.pure_vectors)
        print("pure scalars")
        self.pure_scalars = [vector_to_scalar(*vector) for vector in self.pure_vectors]
        print(self.pure_scalars)
        print("polynomial")
        x = sympy.symbols("x")
        polynomial = sympy.Poly(reduce(lambda a, b: a*b, [x+root for root in self.pure_scalars]))
        self.polynomial = polynomial.coeffs()
        print(self.polynomial)
        print("state")
        self.state = qutip.Qobj(np.array(self.polynomial).T)
        print(self.state)
    
    def hamiltonian_transform(self):
        propagator = scipy.linalg.expm(-2*math.pi*complex(0,1)*self.energy.full()*self.dt)
        self.state = qutip.Qobj(propagator.dot(self.state.full()))
        
    def evolve(self):
        self.mobius_transform()
        self.shatter()
        self.t += self.dt
        
################
sphere = Sphere(8)
################

vpython.scene.range = 1
vpython.scene.forward = vpython.vector(random.random(), random.random(), random.random())
vpython.scene.lights = []
#vsphere = vpython.sphere(pos=vpython.vector(0,0,0), radius=1, color=vpython.color.blue, opacity=0.5)
vpure_stars = [vpython.sphere(pos=vpython.vector(*vector),\
                              radius=0.1, color=vpython.color.white,\
                              opacity=1.0, emissive=True) 
                                     for vector in sphere.pure_vectors]
vimpure_stars = [vpython.sphere(pos=vpython.vector(*vector),\
                                radius=0.1, color=vpython.color.red,\
                                opacity=1.0, emissive=True) 
                                     for vector in sphere.impure_vectors]

def draw_stars():
    vpython.rate(15)
    for i in range(len(vpure_stars)):
        vpure_stars[i].pos = vpython.vector(*sphere.pure_vectors[i])
        vpure_stars[i].color = vpython.color.hsv_to_rgb(vpython.vector(sphere.pure_energy**2,1,1))
        #vsphere.color = vpython.color.hsv_to_rgb(vpython.vector(sphere.pure_energy**2,1,1))
    for i in range(len(vimpure_stars)):
         vimpure_stars[i].pos = vpython.vector(*sphere.impure_vectors[i])
         vimpure_stars[i].color = vpython.color.hsv_to_rgb(vpython.vector(sphere.impure_energies[i]**2,1,1))

def click(event):
    global sphere
    global vpure_stars
    global vimpure_stars
    star = vpython.scene.mouse.pick
    matrix = None
    if star in vpure_stars:
        i = vpure_stars.index(star)
        matrix = sphere.pure_matrices[i]
    elif star in vimpure_stars:
        i = vimpure_stars.index(star)
        matrix = sphere.impure_matrices[i]
    else:
        return
    sphere.mobius = matrix
    
vpython.scene.bind('click', click)
 
while True:
    draw_stars()
    sphere.evolve()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

pure_matrices
[array([[mpc(real='0.41774864109905607', imag='0.0'),
        mpc(real='-0.9084389828625018', imag='0.014989572229890303')],
       [mpc(real='-0.9084389828625018', imag='-0.014989572229890303'),
        mpc(real='-0.41774864109905607', imag='0.0')]], dtype=object), array([[mpc(real='-0.7707204174101101', imag='0.0'),
        mpc(real='-0.045904547124928009', imag='0.63551775013798062')],
       [mpc(real='-0.045904547124928009', imag='-0.63551775013798062'),
        mpc(real='0.7707204174101101', imag='0.0')]], dtype=object), array([[mpc(real='0.44543774794477414', imag='0.0'),
        mpc(real='0.85092059029808786', imag='-0.27842299063231368')],
       [mpc(real='0.85092059029808786', imag='0.27842299063231368'),
        mpc(real='-0.44543774794477414', imag='0.0')]], dtype=object), array([[mpc(real='-0.078593436164296482', imag='0.0'),
        mpc(real='-0.71004478997923526', imag='-0.699756720593116')],
       [mpc(real='-0.71004478997923526', imag='0.699756720593116

Exception: Operator and state do not have same tensor structure: [8] and [1]